In [1]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

True

In [2]:
file = 'users_ai-for-earthquake-response_12.xlsx'
users = pd.read_excel(file)

# users

In [3]:
users['Applicable?'].unique()

array(['Yes', 'No'], dtype=object)

In [4]:
valid_users = users[users['Applicable?'] == 'Yes']

valid_users.shape

(110, 9)

In [5]:
invalid_users = users[users['Applicable?'] == 'No']

invalid_users.shape

(4, 9)

In [6]:
api_url = 'https://api.eotdl.com'
dataset_ids = ['685fdfd4acc05f06ec9919e5', '687b5f9825bb57c11769bb8d'] 
# ai-for-earthquake-response (slim and full)

eotdl_responses_slim, eotdl_responses_full = [], []
for _, user in tqdm(users.iterrows(), total=len(users)):
    if user['Applicable?'] != 'Yes':
        eotdl_responses_slim.append('not added')
        eotdl_responses_full.append('not added')        
        continue
    # print(user.Email)
    for ix, dataset_id in enumerate(dataset_ids):
        response = requests.patch(
            f"{api_url}/datasets/{dataset_id}/allow-user", 
            headers={"X-API-Key": os.getenv('KEY')},
            json={"email": user.Email}
        )
        # print(response.status_code)
        # print(response.json())
        if response.status_code == 200:
            if ix == 0:
                eotdl_responses_slim.append('added')
            else:
                eotdl_responses_full.append('added')
        else:
            if ix == 0:
                eotdl_responses_slim.append(response.json()['detail'])
            else:
                eotdl_responses_full.append(response.json()['detail'])
    # break

100%|██████████| 114/114 [00:30<00:00,  3.75it/s]


In [ ]:
users['EOTDL Response (slim)'] = eotdl_responses_slim
users['EOTDL Response (full)'] = eotdl_responses_full

# users

In [9]:
users.to_excel(file.replace('.xlsx', '_eotdl.xlsx'), index=False)

In [ ]:
# query = {"email": {"$in": users['Email'].tolist()}}
# query

In [ ]:
# _users = pd.read_csv('users2.csv')
# _users

In [ ]:
# Find all users in 'users' not in '_users' by email
# missing_users = users[~users['Email'].isin(_users['email'])]
# missing_users


In [ ]:
# missing_users.to_excel('missing_users.xlsx', index=False)

In [12]:
# _users.id.values